In [2]:
import pandas as pd
import numpy as np
import os
from keras.preprocessing.image import Iterator,load_img
from keras import layers
from keras import models
from keras import optimizers
from util.neuralnet import neuralnet
from skimage import img_as_float
from skimage import io

Using TensorFlow backend.


ModuleNotFoundError: No module named 'util'

In [ ]:
class ValIterator(Iterator):
    """Class to iterate A and B images at the same time."""

    def __init__(
                 self, 
                 dir_dataset, 
                 csv_v_filename_prefix = "Fovea_locations_v.csv", 
                 dir_Validation ="Validation", 
                 dir_Glaucoma = "Glaucoma",
                 dir_NonGlaucoma = "Non-Glaucoma",
                 target_size=(150,150),
                 batch_size=32, 
                 shuffle=True, 
                 seed = None
                ):
        
        self.target_size=target_size
        
        # Carrega nome ficheiros das imagens de retina
        ## Validation
        self.path_Validation = os.path.join(dir_dataset, dir_Validation)
        
        self.ficheiros_Validation = set(x for x in os.listdir(self.path_Validation))
        print ("Numero de ficheiros Validation:", len(self.ficheiros_Validation))
        
        self.n=len(self.ficheiros_Validation)
            
        self.a_fnames = sorted(os.listdir(self.path_Validation))
        
        self.filenames=list(self.ficheiros_Validation)

        # Carega ficheiro CSV com posicao da Fove
        ## Validation
        
        path_csv_v = os.path.join(dir_dataset, csv_v_filename_prefix)

        self.data_Validation = pd.read_csv(path_csv_v)
        
        self.Fovea_X_Validation=self.data_Validation.iloc[:,3]
        self.Fovea_Y_Validation=self.data_Validation.iloc[:,4]
        
        self.img_shape_a = self._get_img_shape(self.target_size, 1)
        self.img_shape_b = self._get_img_shape(self.target_size, 1)
        
        
        super(ValIterator, self).__init__(len(self.a_fnames), batch_size,
                                               shuffle, seed)

In [ ]:
def load_imgs(self, idx):
        """Get a pair of images with index idx."""
        a_fname = self.filenames[idx]

        a = load_img(os.path.join(self.path_Validation, a_fname),
                     grayscale=True,
                     target_size=self.target_size)
        
        a = np.array(a)
  
        if(len(a.shape)<3):
            a = a.reshape(self.img_shape_a)

        return a

In [ ]:
def _get_img_shape(self, size, ch=1):

        img_shape = size + (ch,)
    
        return img_shape

In [ ]:
def load_to_mem(self):
        """Load images to memory."""
        self.a = np.zeros((self.N,) + self.img_shape_a)

        for idx in range(self.N):
            ai = self.load_imgs(idx)
            self.a[idx] = ai  

In [ ]:
def next(self):
        """Get the next pair of the sequence."""

        # Lock the iterator when the index is changed.
        with self.lock:
            index_array = next(self.index_generator)    
        current_batch_size = len(index_array)
        
        batch_a = np.zeros((current_batch_size,) + self.img_shape_a)
        batch_b = np.zeros((current_batch_size,))
        batch_c = np.zeros((current_batch_size,))


        for i, j in enumerate(index_array):
            a_img= self.load_imgs(j)
            #if(np.max(batch_a)>1.):
                #a_img=a_img/255.

            batch_a[i] = a_img
            batch_b[i] = self.Fovea_X_Validation[j]
            batch_c[i] = self.Fovea_Y_Validation[j]
        
        batch_a=batch_a/255.
        batch_b=batch_b-np.mean(batch_b)/np.std(batch_b)
        batch_c=batch_c-np.mean(batch_c)/np.std(batch_c)
        return [batch_a,batch_b,batch_c]


In [ ]:
def generator(self):
        while 1:
            x,y,z = self.next()
            #print x.shape,y.shape
            yield x,y,z
            

In [ ]:
class TrainIterator(Iterator):
    """Class to iterate A and B images at the same time."""

    def __init__(
                 self, 
                 dir_dataset, 
                 csv_t_filename_prefix = "Fovea_locations_t.csv", 
                 dir_Validation ="Validation", 
                 dir_Training = "Training",
                 dir_Glaucoma = "Glaucoma",
                 dir_NonGlaucoma = "Non-Glaucoma",
                 target_size=(150,150),
                 batch_size=32, 
                 shuffle=True, 
                 seed = None
                ):
    
        
        self.target_size=target_size
        

        ## Training
        self.path_Training = os.path.join(dir_dataset, dir_Training)
        
        self.ficheiros_Training = set(x for x in os.listdir(self.path_Training))
        print ("Numero de ficheiros Training:", len(self.ficheiros_Training))
        
        self.n=len(self.ficheiros_Training)
        
        self.a_fnames = sorted(os.listdir(self.path_Training))

        self.filenames=list(self.ficheiros_Training)       
        # Carega ficheiro CSV com posicao da Fove
        ## Validation
        
        
        ## Training
        
        path_csv_t = os.path.join(dir_dataset,csv_t_filename_prefix)

        self.data_Training = pd.read_csv(path_csv_t)
        
        self.Fovea_X_Training=self.data_Training.iloc[:,2]
        self.Fovea_Y_Training=self.data_Training.iloc[:,3]
        
        self.img_shape_a = self._get_img_shape(self.target_size, 1)
        self.img_shape_b = self._get_img_shape(self.target_size, 1)
        
        super(TrainIterator, self).__init__(len(self.a_fnames), batch_size,
                                               shuffle, seed)

In [ ]:
def _get_img_shape(self, size, ch=1):

    img_shape = size + (ch,)

    return img_shape

In [ ]:
def load_imgs(self, idx):
        """Get a pair of images with index idx."""
        a_fname = self.filenames[idx]

        a = load_img(os.path.join(self.path_Training, a_fname),
                     grayscale=True,
                     target_size=self.target_size)
        
        a = np.array(a)
  
        if(len(a.shape)<3):
            a = a.reshape(self.img_shape_a)

        return a